## Configure a server
after you created your server, this notebook will handle stitching everything together, we will start by:
- Get credentials for our object store buckets
- Run our containerized environment (Jupyter and MLflow)
###  Prerequisites
- This notebook assumes that you are logged into Chameleon JupyterHub and are running the cells from within that environment
- you have created a server and its ACTIVE in status 

## Retrieve server

We will start by getting our server object in this notebook using python-chi 

In [ ]:
from chi import server, context
import getpass
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [ ]:
server_name = None # replace with the server name you want to select
project_name = "MLflow_amd"

In [ ]:
def get_active_server(server_name=None, project_name=None):
    all_servers = server.list_servers()

    if server_name:
        match = next((s for s in all_servers if s.name == server_name), None)
        if not match:
            raise ValueError(f"no server found with name '{server_name}'")
        return match

    matching = [s for s in all_servers if s.name.startswith(project_name)]

    if not matching:
        raise ValueError(f"no server found starting with '{project_name}'")

    if len(matching) > 1:
        print("multiple servers found:")
        for s in matching:
            print(f" - {s.name} (status: {s.status})")
        raise ValueError("set 'server_name' to select the correct server.")

    return matching[0]


In [ ]:
s = get_active_server(server_name, project_name)
s.show()

## Generate environment variables 

after configuring our server and before running our containerized environment we need to generate environment variables to be included in our MLflow server so it can communicate to the S3_Buckets we created earlier, 
 the variables are going to be stored on `.env` file on our home directory on the compute instance 


### HuggingFace setup 




In [ ]:

# this is a hidden input, to make sure we are not pushing this to remote repository with the token visiable
hf_token = getpass.getpass("enter you Hugging face token 🤗 : ")  
# generating the .env file
s.execute(f"bash ./MLflow_amd/scripts/generate_env.sh '{hf_token}'")  # passing hf_token as an argument



## Run our containerized environment 

now we will pass the generated environment variables stored on .env file and run our docker-compose.yml to have our Jupyter lab and MLflow server up and running.

In [ ]:
s.execute("docker compose --env-file ~/.env -f MLflow_amd/docker/docker-compose.yml up -d --build")

## Login to Jupyter Lab and MLFlow UI
Now that we have our containers running, we need to login, for Jupyter run the following commands to get the URL

In [ ]:
# Jupyter Lab URL open the URL in your browser.
s.execute('docker exec jupyter jupyter server list | tail -n 1 | cut -f1 -d" " | sed "s/localhost/$(curl -s ifconfig.me)/"')

In [ ]:
# MLflow server URL
s.execute('echo "http://$(curl -s ifconfig.me):$(docker port mlflow 8000 | cut -d":" -f2)"')

## Next Steps & Further Actions

You've completed the main workflow in this notebook! 
There’s more you can do if you want to explore further:

- Stop your containerized environment safely when you're done working.
- logging into GitHub from the contrainer 

For detailed instructions on these actions, please refer to the [**project README.md file**](../README.md)
